In [65]:
import cv2
import numpy as np

In [66]:
video_path = r'D:\MAPUA\Thesis\Video_Sample1\Video1.mp4'

# Parameters for Shi-Tomasi corner detection
feature_params = dict(maxCorners=1000, qualityLevel=0.01, minDistance=10, blockSize=3)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15), maxLevel=3, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Read the video
cap = cv2.VideoCapture(video_path)
# Create a background subtractor
background_subtractor = cv2.createBackgroundSubtractorMOG2()

# Read the first frame
ret, prev_frame = cap.read()
prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

# Detect the arm region using color segmentation
hsv_frame = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2HSV)
lower_skin = np.array([0, 20, 70], dtype=np.uint8)
upper_skin = np.array([20, 255, 255], dtype=np.uint8)
arm_mask = cv2.inRange(hsv_frame, lower_skin, upper_skin)

# Apply morphological operations to refine the mask
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
arm_mask = cv2.morphologyEx(arm_mask, cv2.MORPH_OPEN, kernel)

# Detect features in the arm region
prev_corners = cv2.goodFeaturesToTrack(prev_gray, mask=arm_mask, **feature_params)

# Create an empty mask for drawing purposes
mask = np.zeros_like(prev_frame)

while True:
    # Read the current frame
    ret, frame = cap.read()
    if not ret:
        break

    # Apply background subtraction
    fg_mask = background_subtractor.apply(frame)

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Perform bitwise AND operation to extract arm regions
    arm_mask = cv2.bitwise_and(fg_mask, gray)

    # Track features using Lucas-Kanade optical flow
    next_corners, status, err = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_corners, None, **lk_params)

    # Filter out invalid and low-quality tracks
    good_old = prev_corners[status == 1]
    good_new = next_corners[status == 1]
    good_err = err[status == 1]

    # Refine feature set based on error threshold
    good_old = good_old[good_err < 10]
    good_new = good_new[good_err < 10]

    # Draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        mask = cv2.line(mask, (int(a), int(b)), (int(c), int(d)), (0, 255, 0), 2)
        frame = cv2.circle(frame, (int(a), int(b)), 5, (0, 255, 0), -1)

    # Overlay the optical flow tracks on the frame
    img = cv2.add(frame, mask)

    # Display the resulting image
    cv2.imshow('KLT Tracker', img)
    if cv2.waitKey(int(1000 / cap.get(cv2.CAP_PROP_FPS))) & 0xFF == ord('q'):
        break

    # Update the previous frame, corners, and arm mask
    prev_gray = gray.copy()
    prev_corners = good_new.reshape(-1, 1, 2)
    arm_mask = np.zeros_like(gray)
    arm_mask = cv2.bitwise_or(arm_mask, arm_mask)

# Release resources
cap.release()
cv2.destroyAllWindows()

Arm Tracking Config

In [67]:
video_path = r'D:\MAPUA\Thesis\Video_Sample1\ArmTrack1.mp4'

# Parameters for skin color segmentation
lower_skin = np.array([0, 20, 70], dtype=np.uint8)
upper_skin = np.array([20, 255, 255], dtype=np.uint8)

# Parameters for Shi-Tomasi corner detection
feature_params = dict(maxCorners=1000, qualityLevel=0.01, minDistance=10, blockSize=3)

# Parameters for Lucas-Kanade optical flow
lk_params = dict(winSize=(15, 15), maxLevel=3, criteria=(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

# Read the video
cap = cv2.VideoCapture('D:\MAPUA\Thesis\Video_Sample1\ArmTrack1.mp4')

# Create a background subtractor
background_subtractor = cv2.createBackgroundSubtractorMOG2()

# Create an empty mask for drawing purposes
mask = np.zeros((1, 1), dtype=np.uint8)

# Initialize variables for KLT tracking
prev_gray = None
prev_points = None

# Define a region of interest (ROI) for arm tracking
roi_x, roi_y, roi_width, roi_height = 100, 100, 200, 200

while True:
    # Read the current frame
    ret, frame = cap.read()
    if not ret:
        break

    # Apply background subtraction
    fg_mask = background_subtractor.apply(frame)

    # Convert the frame to HSV color space
    hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

    # Perform skin color segmentation to isolate arm regions
    arm_mask = cv2.inRange(hsv_frame, lower_skin, upper_skin)

    # Apply morphological operations to refine the mask
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    arm_mask = cv2.morphologyEx(arm_mask, cv2.MORPH_OPEN, kernel)

    # Define the ROI within the arm mask
    roi = arm_mask[roi_y:roi_y + roi_height, roi_x:roi_x + roi_width]

    # Detect corners within the ROI using Shi-Tomasi corner detection
    corners = cv2.goodFeaturesToTrack(roi, mask=None, **feature_params)
    if corners is not None:
        # Adjust corner coordinates based on ROI position
        corners = corners + np.array([roi_x, roi_y])

        # Convert the frame to grayscale
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        if prev_gray is None:
            # First frame, initialize previous frame and points
            prev_gray = gray.copy()
            prev_points = corners

        else:
            # Track features using Kanade-Lucas-Tomasi (KLT) tracker
            next_points, status, _ = cv2.calcOpticalFlowPyrLK(prev_gray, gray, prev_points, None, **lk_params)

            # Filter out invalid and low-quality tracks
            good_prev = prev_points[status == 1]
            good_next = next_points[status == 1]

            # Draw optical flow lines on the frame
            for (prev_x, prev_y), (next_x, next_y) in zip(good_prev, good_next):
                cv2.arrowedLine(frame, (int(prev_x), int(prev_y)), (int(next_x), int(next_y)), (0, 255, 0), 2)

            # Display the resulting image
            cv2.imshow('Arm Tracking', frame)
            if cv2.waitKey(int(1000 / cap.get(cv2.CAP_PROP_FPS))) & 0xFF == ord('q'):
                break

            # Update the previous frame and points
            prev_gray = gray.copy()
            prev_points = good_next

# Release resources
cap.release()
cv2.destroyAllWindows()